# Classification and detection model - Version 2

The classification backbone of the model is Resnet network along with the detection activation layers

## Building the model

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K
 
tf.config.run_functions_eagerly(True)

In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16') 
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = keras.layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c', dtype=policy)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16') 

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = keras.layers.Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', 
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c', dtype=policy)(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1',
                                     kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '1', dtype=policy)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


def build_model(input_shape, out_type, number_of_attributes):

    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16')  # sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure

    # Define the input as a tensor with shape input_shape
    X_input = keras.layers.Input(input_shape)

    # Zero-Padding
    X = keras.layers.ZeroPadding2D((3, 3), dtype=policy)(X_input)

    # Stage 1
    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), name='conv1',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), dtype=policy)(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[16, 16, 32], stage=2, block='a', s=1)
    X = identity_block(X, 3, [16, 16, 32], stage=2, block='b')
    X = identity_block(X, 3, [16, 16, 32], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[32, 32, 64], stage=3, block='a', s=2)
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='b')
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='c')
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[64, 64, 128], stage=4, block='a', s=2)
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='b')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='c')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='d')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='e')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 256], stage=5, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 256], stage=5, block='b')
    X = identity_block(X, 3, [128, 128, 256], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = keras.layers.AveragePooling2D((2, 2), name="avg_pool")(X)

    X = keras.layers.Conv2D(16, 1, activation="selu", kernel_initializer="lecun_normal", dtype=policy)(X)
    #new
    X = keras.layers.Conv2DTranspose(1, 3, activation='linear', dtype=policy)(X)
    #new
    X = keras.layers.Conv2DTranspose(1, 3, activation='linear', dtype=policy)(X)
    #X = keras.layers.Conv2D(1, 1, activation='linear')(X)
    X_map = keras.activations.sigmoid(X)

    ### END CODE HERE ###

    # output layer

    # new - changed X to X_map
    X = keras.layers.Flatten()(X_map)
    y = keras.layers.Dense(number_of_attributes, activation='softmax', name='fc' + str(number_of_attributes),
                           kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)

    optimizer = keras.optimizers.SGD(lr = 0.0001, momentum=0.01, nesterov=True)
    metrics = [
        keras.metrics.CategoricalAccuracy(name='accuracy'),
        #keras.metrics.Precision(name='precision'),
        #keras.metrics.Recall(name='recall'),
        #keras.metrics.AUC(name='auc'),
    ]

    if out_type == "classify":
        model = keras.models.Model(inputs=X_input, outputs=y)
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=metrics)
        
    if out_type == "map":
        model = keras.models.Model(inputs=X_input, outputs=X_map)
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=metrics)

    return model


## CASTING DATA

### Pre-processing


In [ ]:
dataset_dir = '/content/drive/MyDrive/Major Project/Datasets/datasets'

filename = 'steel_defect_cls_products.tar.xz'
tar_file = os.path.join(dataset_dir, filename)

my_tar = tarfile.open(tar_file)
my_tar.extractall(dataset_dir) # specify which folder to extract to
my_tar.close()

In [ ]:
import os
import tarfile
import shutil
import numpy as np
from tqdm.notebook import tqdm
import random

dataset_dir = '/content/drive/MyDrive/Major Project/Datasets/datasets'

model_dir = '/content/drive/MyDrive/Major Project/Models/V2'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
dataset = os.path.join(dataset_dir, 'casting_dataset')

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=10,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            brightness_range = [0.5,1.0],
                            zoom_range=0.1,
                            rescale=1./255,
                            fill_mode="nearest",
                            cval=1.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.1,
                            dtype='float64',
                        )


In [ ]:
batch_size = 64

train_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training',
                                                shuffle=True) 
                                            
val_generator = generator.flow_from_directory( os.path.join(dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation',
                                                shuffle=True) 


In [ ]:
attribute_to_idx = train_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

number_of_attributes = len(idx_to_attribute)

In [ ]:
count_dict = {}

for category in os.listdir(os.path.join(dataset, 'train')):

    count_dict[category] = len(os.listdir(os.path.join(dataset, 'train', category)))

print(count_dict)

cutoff = 0

if len(count_dict) == 2:

    cutoff = min(count_dict.values())

if len(count_dict) > 2:

    cutoff = np.median(list(count_dict.values()))

print(cutoff)

#### Balancing dataset

In [ ]:
new_dataset = os.path.join(dataset_dir, 'casting_dataset_balanced')

if not os.path.exists(new_dataset):
    os.mkdir(new_dataset)

for split in os.listdir(dataset):

    print(" ", split)
    split_path = os.path.join(dataset, split)
    new_split_path = os.path.join(new_dataset, split)

    if not os.path.exists(new_split_path):
        os.mkdir(new_split_path)

    for category in os.listdir(split_path):

        print("     ", category)
        
        category_path = os.path.join(split_path, category)
        new_category_path = os.path.join(new_split_path, category)

        if not os.path.exists(new_category_path):
            os.mkdir(new_category_path)
        
        images = os.listdir(category_path)

        for i in range(2):
            random.shuffle(images)

        if len(images) > cutoff:

            images = images[:cutoff]
        
        for image in tqdm(images):

            source = os.path.join(category_path, image)
            destination = os.path.join(new_category_path, image)

            shutil.copyfile(source, destination)


In [ ]:
batch_size = 64

new_dataset = os.path.join(dataset_dir, 'casting_dataset_balanced')

train_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training',
                                                shuffle=True) 
                                            
val_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation',
                                                shuffle=True) 


In [ ]:
attribute_to_idx = train_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

number_of_attributes = len(idx_to_attribute)

### Training model

In [ ]:
map_model = build_model("map", number_of_attributes)
model = build_model("classify", number_of_attributes)
model.summary()

In [ ]:
epochs = 50

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, min_lr = 1e-7, factor=0.5, verbose=1)

history = model.fit(train_generator, 
                    epochs= epochs,
                    validation_data = val_generator,
                    verbose=1,
                    callbacks=[reduce_lr]) 

In [ ]:
model.save(os.path.join(model_dir, 'casting_V1.h5'))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['lr'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.show()

plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.show()

plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.show()

### Testing the model

In [ ]:
model = keras.models.load_model(os.path.join(model_dir, 'casting_V2.h5'))
model.summary()

In [ ]:
batch_size = 64

test_generator = generator.flow_from_directory( os.path.join(new_dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical') 

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
scores_keys = ['loss', 'accuracy', 'precision', 'recall', 'auc']

for key,score in zip(scores_keys, scores):

    print(key, ':', score)

"""
50 eps

loss : 0.01628803461790085
accuracy : 0.9959127902984619
precision : 0.9959127902984619
recall : 0.9959127902984619
auc : 0.9985383749008179

"""

#### Visualise the results

In [ ]:
from google.colab.patches import cv2_imshow

for idx, (images, output) in enumerate(test_generator):

    if idx == 2:
        break

    for i in range(batch_size):

        image = images[i]

        cv2_imshow(image * 255)

        preds = model.predict(np.expand_dims(image, axis=0))

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
    idx += 1

#### Visualise activation maps


In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, model, last_conv_layer, classifier_layer
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    x = keras.layers.Flatten()(x)
    x = classifier_layer(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm

last_conv_layer = model.layers[-4]
classifier_layer = model.layers[-1]

for idx, (images, output) in enumerate(test_generator):

    if idx == 1:
        break

    for i in range(batch_size):
        img = images[i] * 255

        image = np.expand_dims(images[i], axis=0)

        preds = model.predict(image)

        actual_value = np.argmax(output[i], axis=0)

        predicted_value = np.argmax(preds, axis=1)[0]

        confidence = preds[0][predicted_value] * 100

        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(
            image, model, last_conv_layer, classifier_layer
        )


        # We rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # We use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # We use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # We create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * 0.6 + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        plt.figure(figsize=(5,5))
        plt.imshow(superimposed_img)
        plt.show()

        print("Confidence", confidence, "%")
        print("Actual_value", idx_to_attribute[int(actual_value)])
        print("Predicted value", idx_to_attribute[int(predicted_value)])
    
        
    idx += 1

#### Visualise detection results

In [ ]:
batch_size = 1

test_generator = generator.flow_from_directory( os.path.join(new_dataset, 'test'), 
                                                target_size=(224, 224), 
                                                batch_size=batch_size,
                                                class_mode='categorical') 
disp_images = []

pos_idx = 0
neg_idx = 0

for image, actual in test_generator:

    output = np.argmax(actual, axis=1)[0]
    
    if output == 0 and pos_idx < 3:
        pos_idx += 1
        disp_images.append(image[0, ...])

    
    if output == 1 and neg_idx <3:
        neg_idx += 1
        disp_images.append(image[0, ...])

    if neg_idx>=3 and pos_idx>=3:
        break

disp_images = np.array(disp_images)
print(disp_images.shape)

In [ ]:
#takes in 6 images and activation maps, reutrns a 3x2 grid of images with box highlighting   
def plot_maps(images, maps, k, dims):
    map_size=dims[0]
    r_size = dims[1]
    r_stride = dims[2]
    colors = ['#ff0000', '#ff0080', '#ff00ff', '#8000ff', '#0080ff', '#00ffff', '#00ff80']
              
    fig, ax = plt.subplots(2, 3, figsize=(20,20))
    for j , (img, mask) in enumerate(zip(images, maps)):
       
        answer = np.argmax(model.predict(np.expand_dims(img, axis=0)), axis=1)[0]
        answer = idx_to_attribute[answer]

        mask = mask.flatten()
        j_a = (j-j%3)//3
        j_b = j%3
        img = np.asarray(img[:,:,0],dtype=np.float32)/255
        ax[j_a][j_b].imshow(img,cmap=plt.get_cmap('gray'))
        
        for i in range(k):
            a_max = np.argmax(mask)
            x_region = a_max%map_size#note, numpy addresses work on arr[y, x, z], compared to image coordinates
            y_region = (a_max-(a_max%map_size))//map_size
            prob = mask[a_max]
            if prob<0.5:break
            x_pixel = r_stride*(x_region)
            y_pixel = r_stride*(y_region)

            #rectangle expects bottom left coordinates. We've generated Top Right
            rectangle = mplot.patches.Rectangle((x_pixel, y_pixel), r_size, r_size, edgecolor=colors[i-1],facecolor="none")
            ax[j_a][j_b].add_patch(rectangle)
            
            font = {'color':colors[i-1], 'size':20}
            ax[j_a][j_b].text(x_pixel,y_pixel,s="{0:.3f}".format(prob), fontdict=font)
            ax[j_a][j_b].text(200,200, s=answer)
            mask[a_max]= 0#to help get the next most maximum
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()   

In [ ]:
map_model = keras.models.Model(inputs=[model.input], 
                               outputs=[model.layers[-3].output])

final_layer = model.layers[-1]

pred_maps = map_model.predict(disp_images)

plot_maps(disp_images, pred_maps, 10,  (11, 32, 20))

## STEEL DEFECT DATA

### Pre-processing

In [ ]:
import os
import tarfile
import shutil
import numpy as np
import random
from tqdm.notebook import tqdm 
import matplotlib.pyplot as plt

dataset_dir = '/content/drive/MyDrive/Major Project/Datasets/datasets'

model_dir = '/content/drive/MyDrive/Major Project/Models/V2'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

#### Balancing the dataset

In [ ]:
dataset = os.path.join(dataset_dir, 'steel_defect_cls_products_balanced')

count_dict = {}

train_dataset = os.path.join(dataset, 'train')

for category in os.listdir(train_dataset):

    category_path = os.path.join(train_dataset, category)

    count_dict[category] = len(os.listdir(category_path))

print(count_dict)
"""
cutoff = 0

if len(count_dict) == 2:
    cutoff = min(count_dict.values())

if len(count_dict) > 2:
    cutoff = int(np.median(list(count_dict.values())))

print(cutoff)
"""

In [ ]:
dataset = os.path.join(dataset_dir, 'steel_defect_cls_products')
new_dataset = os.path.join(dataset_dir, 'steel_defect_cls_products_balanced')

if not os.path.exists(new_dataset):
    os.mkdir(new_dataset)

for split in os.listdir(dataset):

    print(" ", split)
    split_path = os.path.join(dataset, split)
    new_split_path = os.path.join(new_dataset, split)

    if not os.path.exists(new_split_path):
        os.mkdir(new_split_path)

    for category in os.listdir(split_path):

        print("     ", category)
        
        category_path = os.path.join(split_path, category)
        new_category_path = os.path.join(new_split_path, category)

        if not os.path.exists(new_category_path):
            os.mkdir(new_category_path)
        
        images = os.listdir(category_path)

        for i in range(2):
            random.shuffle(images)

        if len(images) > cutoff:

            images = images[:cutoff]
        
        for image in tqdm(images):

            source = os.path.join(category_path, image)
            destination = os.path.join(new_category_path, image)

            shutil.copyfile(source, destination)


#### Creating data generators for training

In [ ]:
import keras

new_dataset = os.path.join(dataset_dir, 'steel_defect_cls_products_balanced')

generator = keras.preprocessing.image.ImageDataGenerator(
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            rescale=1./255,
                            fill_mode="constant",
                            cval=0.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            validation_split=0.1,
                            dtype='float32',
                        )

In [ ]:
batch_size = 64

train_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(200, 700), 
                                                batch_size=batch_size,
                                                class_mode='binary',
                                                subset='training') 
                                            
val_generator = generator.flow_from_directory( os.path.join(new_dataset, 'train'), 
                                                target_size=(200, 700), 
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation') 



In [ ]:
attribute_to_idx = train_generator.class_indices
idx_to_attribute = {value:key for key,value in attribute_to_idx.items()}

print(attribute_to_idx)
print(idx_to_attribute)

number_of_attributes = len(idx_to_attribute)

In [ ]:
a = next(train_generator)

for img in a[1][:10]:
    print(img)
    plt.figure(figsize=(15,10))
    plt.imshow(img)
    plt.show()


### Building the model

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K

tf.config.run_functions_eagerly(True)

In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16') 
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = keras.layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c', dtype=policy)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16') 

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = keras.layers.Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a', 
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c', dtype=policy)(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1',
                                     kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '1', dtype=policy)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


def build_model(input_shape, out_type, number_of_attributes):

    policy = tf.keras.mixed_precision.experimental.Policy(
        'mixed_float16')  # sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure

    # Define the input as a tensor with shape input_shape
    X_input = keras.layers.Input(input_shape)

    # Zero-Padding
    X = keras.layers.ZeroPadding2D((3, 3), dtype=policy)(X_input)

    # Stage 1
    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), name='conv1',
                            kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1', dtype=policy)(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), dtype=policy)(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[16, 16, 32], stage=2, block='a', s=1)
    X = identity_block(X, 3, [16, 16, 32], stage=2, block='b')
    X = identity_block(X, 3, [16, 16, 32], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[32, 32, 64], stage=3, block='a', s=2)
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='b')
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='c')
    X = identity_block(X, 3, [32, 32, 64], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[64, 64, 128], stage=4, block='a', s=2)
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='b')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='c')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='d')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='e')
    X = identity_block(X, 3, [64, 64, 128], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 256], stage=5, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 256], stage=5, block='b')
    X = identity_block(X, 3, [128, 128, 256], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = keras.layers.AveragePooling2D((2, 2), name="avg_pool")(X)

    X = keras.layers.Conv2D(16, 1, activation="selu", kernel_initializer="lecun_normal", dtype=policy)(X)
    #new
    X = keras.layers.Conv2DTranspose(1, 3, activation='linear', dtype=policy)(X)
    #new
    X = keras.layers.Conv2DTranspose(1, 3, activation='linear', dtype=policy)(X)
    #X = keras.layers.Conv2D(1, 1, activation='linear')(X)
    X_map = keras.activations.sigmoid(X)

    ### END CODE HERE ###

    # output layer

    # new - changed X to X_map
    X = keras.layers.Flatten()(X_map)
    y = keras.layers.Dense(number_of_attributes, activation='sigmoid', name='fc' + str(number_of_attributes),
                           kernel_initializer=keras.initializers.glorot_uniform(seed=0), dtype=policy)(X)

    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    metrics = [
        keras.metrics.CategoricalAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
    ]

    if out_type == "classify":
        model = keras.models.Model(inputs=X_input, outputs=y)
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=metrics)
        
    if out_type == "map":
        model = keras.models.Model(inputs=X_input, outputs=X_map)
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=metrics)

    return model


In [ ]:
model = build_model((200,700,3), "classify", number_of_attributes)
model.summary()

### Training the model

In [ ]:
epochs = 50

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, min_lr = 1e-8, factor=0.5, verbose=1)

history = model.fit(train_generator, 
                    epochs= epochs,
                    validation_data = val_generator,
                    verbose=1,
                    callbacks=[reduce_lr],
                   )

In [ ]:
model.save(os.path.join(model_dir, 'steel_V1_100eps.h5'))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['lr'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.show()

plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.show()

plt.plot(history.history['auc'])

plt.plot(history.history['val_auc'])
plt.show()

### Testing the model

In [ ]:
model = keras.models.load_model(os.path.join(model_dir, 'steel_V1_100eps.h5'))

In [ ]:
batch_size = 64

test_generator = generator.flow_from_directory( os.path.join(new_dataset, 'test'), 
                                                target_size=(200, 800), 
                                                batch_size=batch_size,
                                                class_mode='categorical') 

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
scores_keys = ['loss', 'accuracy', 'precision', 'recall', 'auc']

for key,score in zip(scores_keys, scores):

    print(key, ':', score)

"""
100 eps

loss : 0.6535497307777405
accuracy : 0.7710843086242676
precision : 0.7991266250610352
recall : 0.7349397540092468
auc : 0.9266652464866638

"""